# Data Processing Pipeline

This notebook processes the raw Stack Overflow 2025 Developer Survey data following the methodology from the Medium article example.

## Steps:
1. Load raw survey data
2. Filter and clean data
3. Feature engineering
4. Encode categorical variables
5. Create stratified train/test splits
6. Save processed datasets


In [ ]:
import json
import re
from pathlib import Path
from typing import Optional

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Set paths
RAW_DATA = Path("../../data/raw/stack-overflow-developer-survey-2025-2/survey_results_public.csv")
INTERIM_OUTPUT = Path("../../data/interim/so_2025_clean.csv")
PROCESSED_OUTPUT = Path("../../data/processed/so_2025_model_ready.parquet")
TRAIN_SPLIT_OUTPUT = Path("../../data/processed/so_2025_train.parquet")
TEST_SPLIT_OUTPUT = Path("../../data/processed/so_2025_test.parquet")
FEATURE_METADATA = Path("../../data/processed/so_2025_feature_columns.json")

print("Imports successful!")


In [ ]:
# Configuration constants
USE_COLUMNS = [
    "Country",
    "EdLevel",
    "YearsCode",
    "Employment",
    "DevType",
    "ConvertedCompYearly",
    "RemoteWork",
    "Currency",
]

NA_VALUES = ["NA", "Other (please specify):"]

EDUCATION_CATEGORIES = {
    "Less than secondary": [
        "Primary/elementary school",
        "Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)",
    ],
    "Some college": [
        "Some college/university study without earning a degree",
        "Associate degree (A.A., A.S., etc.)",
        "Professional degree (JD, MD, Ph.D, Ed.D, etc.)",
    ],
    "Bachelor's degree": ["Bachelor's degree (B.A., B.S., B.Eng., etc.)"],
    "Master's degree": ["Master's degree (M.A., M.S., M.Eng., MBA, etc.)"],
    "Post-grad": ["Doctoral degree (Ph.D)"],
    "Self-taught/other": [
        "Something else",
        "I prefer not to say",
    ],
}

REMOTE_CATEGORIES = {
    "Remote": "Remote",
    "Hybrid (some in-person, leans heavy to flexibility)": "Hybrid-Flexible",
    "Hybrid (some remote, leans heavy to in-person)": "Hybrid-InPerson",
    "Your choice (very flexible, you can come in when you want or just as needed)": "Hybrid-Choice",
    "In-person": "In-person",
}

EMPLOYMENT_ALLOWED = {
    "Employed",
    "Independent contractor, freelancer, or self-employed",
}

print("Configuration loaded!")


In [ ]:
# Helper functions
def simplify_edlevel(value: Optional[str]) -> Optional[str]:
    if pd.isna(value):
        return None
    for label, raw_values in EDUCATION_CATEGORIES.items():
        if value in raw_values:
            return label
    return value

def parse_years_code(value: Optional[str]) -> Optional[float]:
    if pd.isna(value):
        return None
    if isinstance(value, (int, float)):
        return float(value)
    value = value.strip()
    if value.startswith("Less than"):
        return 0.5
    if value.startswith("More than"):
        match = re.search(r"(\d+)", value)
        if match:
            return float(match.group(1))
        return None
    try:
        return float(value)
    except ValueError:
        return None

def normalize_devtype(value: Optional[str]) -> Optional[str]:
    if pd.isna(value):
        return None
    return value.split(";")[0].strip()

def normalize_remote(value: Optional[str]) -> Optional[str]:
    if pd.isna(value):
        return None
    return REMOTE_CATEGORIES.get(value, value)

print("Helper functions defined!")


## Step 1: Load Raw Dataset


In [ ]:
# Load raw dataset
if not RAW_DATA.exists():
    raise FileNotFoundError(f"Raw dataset not found at {RAW_DATA}")

df_raw = pd.read_csv(RAW_DATA, usecols=USE_COLUMNS, na_values=NA_VALUES)
print(f"Raw dataset loaded: {len(df_raw):,} rows, {len(df_raw.columns)} columns")
print(f"\nColumns: {list(df_raw.columns)}")
print(f"\nFirst few rows:")
df_raw.head()


In [ ]:
# Check missing values
print("Missing values per column:")
print(df_raw.isna().sum())
print(f"\nMissing value percentages:")
print((df_raw.isna().sum() / len(df_raw) * 100).round(2))


## Step 2: Clean Dataset


In [ ]:
# Start cleaning
df_clean = df_raw.copy()

# Filter by employment status
print(f"Before employment filter: {len(df_clean):,} rows")
df_clean = df_clean[df_clean["Employment"].isin(EMPLOYMENT_ALLOWED)]
print(f"After employment filter: {len(df_clean):,} rows")

# Apply transformations
df_clean["EdLevelSimplified"] = df_clean["EdLevel"].apply(simplify_edlevel)
df_clean["YearsCodeNum"] = df_clean["YearsCode"].apply(parse_years_code)
df_clean["DevTypePrimary"] = df_clean["DevType"].apply(normalize_devtype)
df_clean["RemoteCategory"] = df_clean["RemoteWork"].apply(normalize_remote)

# Rename salary column
df_clean = df_clean.rename(columns={"ConvertedCompYearly": "CompYearlyUSD"})

print(f"\nAfter transformations: {len(df_clean):,} rows")


In [ ]:
# Drop rows with missing critical values
print(f"Before dropping NA: {len(df_clean):,} rows")
df_clean = df_clean.dropna(
    subset=[
        "Country",
        "CompYearlyUSD",
        "EdLevelSimplified",
        "YearsCodeNum",
        "DevTypePrimary",
    ]
)
print(f"After dropping NA: {len(df_clean):,} rows")

# Filter salary range (USD 1k - 600k)
print(f"\nBefore salary filter: {len(df_clean):,} rows")
df_clean = df_clean[(df_clean["CompYearlyUSD"] >= 1_000) & (df_clean["CompYearlyUSD"] <= 600_000)]
print(f"After salary filter: {len(df_clean):,} rows")


In [ ]:
# Select final columns and add log-transformed salary
selected_columns = [
    "CompYearlyUSD",
    "Country",
    "EdLevelSimplified",
    "YearsCodeNum",
    "Employment",
    "DevTypePrimary",
    "RemoteCategory",
]

df_clean = df_clean[selected_columns].reset_index(drop=True)
df_clean["SalaryLog10"] = np.log10(df_clean["CompYearlyUSD"])

print(f"Final clean dataset: {len(df_clean):,} rows, {len(df_clean.columns)} columns")
print(f"\nColumns: {list(df_clean.columns)}")
df_clean.head()


In [ ]:
# Save clean dataset
INTERIM_OUTPUT.parent.mkdir(parents=True, exist_ok=True)
df_clean.to_csv(INTERIM_OUTPUT, index=False)
print(f"Clean dataset saved to {INTERIM_OUTPUT}")


## Step 3: Encode Features


In [ ]:
# Prepare features for encoding
feature_cols = [
    "Country",
    "EdLevelSimplified",
    "YearsCodeNum",
    "DevTypePrimary",
    "RemoteCategory",
]

cat_cols = [
    "Country",
    "EdLevelSimplified",
    "DevTypePrimary",
    "RemoteCategory",
]

num_cols = ["YearsCodeNum"]
target = df_clean["CompYearlyUSD"].copy()

# One-hot encode categorical variables
encoded = pd.get_dummies(df_clean[cat_cols], drop_first=True)
encoded[num_cols] = df_clean[num_cols]
encoded = encoded.fillna(0.0)

feature_list = encoded.columns.tolist()

print(f"Encoded features: {len(feature_list)} columns")
print(f"Sample feature names: {feature_list[:10]}")
encoded.head()


In [ ]:
# Create processed dataset with target
processed = encoded.copy()
processed["CompYearlyUSD"] = target

PROCESSED_OUTPUT.parent.mkdir(parents=True, exist_ok=True)
processed.to_parquet(PROCESSED_OUTPUT, index=False)

# Save feature metadata
FEATURE_METADATA.write_text(json.dumps({"feature_columns": feature_list}, indent=2))

print(f"Encoded dataset saved to {PROCESSED_OUTPUT}")
print(f"Feature metadata saved to {FEATURE_METADATA}")


## Step 4: Create Stratified Train/Test Split

For regression problems, we use stratified splitting by binning the target variable to ensure similar salary distributions in train and test sets.


In [ ]:
# Create bins for stratified splitting
# Use quantiles to create balanced bins
n_bins = 10
target_binned = pd.qcut(target, q=n_bins, labels=False, duplicates='drop')

print(f"Target distribution across bins:")
print(pd.Series(target_binned).value_counts().sort_index())

# Perform stratified split
X_train, X_test, y_train, y_test = train_test_split(
    encoded,
    target,
    test_size=0.2,
    random_state=450,
    shuffle=True,
    stratify=target_binned  # Stratify by binned target
)

print(f"\nTrain set: {len(X_train):,} samples")
print(f"Test set: {len(X_test):,} samples")
print(f"\nTrain target stats:")
print(y_train.describe())
print(f"\nTest target stats:")
print(y_test.describe())


In [ ]:
# Create train and test datasets with target
train_split = X_train.copy()
test_split = X_test.copy()
train_split["CompYearlyUSD"] = y_train
test_split["CompYearlyUSD"] = y_test

# Save splits
TRAIN_SPLIT_OUTPUT.parent.mkdir(parents=True, exist_ok=True)
train_split.to_parquet(TRAIN_SPLIT_OUTPUT, index=False)
test_split.to_parquet(TEST_SPLIT_OUTPUT, index=False)

# Also save as CSV for easier access
train_split.to_csv(TRAIN_SPLIT_OUTPUT.with_suffix('.csv'), index=False)
test_split.to_csv(TEST_SPLIT_OUTPUT.with_suffix('.csv'), index=False)

print(f"Train split saved to {TRAIN_SPLIT_OUTPUT}")
print(f"Test split saved to {TEST_SPLIT_OUTPUT}")
print(f"\n✅ Processing complete!")
